In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
# %%
import torch
import string

In [3]:
!pip install sentencepiece

In [4]:
!pip install transformers

In [5]:
cd drive/My Drive/Colab Notebooks/experiments

/content/drive/My Drive/Colab Notebooks/experiments


##BERT

In [6]:
from transformers import BertTokenizer, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
cd drive/My Drive/Colab Notebooks/experiments

[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/experiments'
/content/drive/My Drive/Colab Notebooks/experiments


In [8]:
# Use a subset for quick experiments
#data = data[:10000]

from sklearn.model_selection import train_test_split as tts
import pandas as pd
data = pd.read_csv("data/moh-x.csv")

# Split to train, val and test
text, test_df = tts(data[["sentence", "arg1", "verb", "label"]], random_state=42, test_size=0.1)
text, val = tts(text, random_state=42, test_size=test_df.shape[0])

In [9]:
text = text["sentence"]

In [10]:
#text.to_csv(r'mohx.txt', header=None, index=None, sep=' ', mode='a')

In [11]:
with open('data/mohx.txt', 'r') as fp:
    text = fp.read().split('\n')

In [12]:
text[:5]

['" I ca n\'t buy this story ."',
 '" European children learn the breast stroke ; they often do n\'t know how to crawl ."',
 '" He shed his image as a pushy boss ."',
 '" She devoured his novels ."',
 '" Age and experience mellowed him over the years ."']

In [13]:
inputs = tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [14]:
inputs

{'input_ids': tensor([[  101,  1000,  1045,  ...,     0,     0,     0],
        [  101,  1000,  2647,  ..., 13529,  1012,   102],
        [  101,  1000,  2002,  ...,     0,     0,     0],
        ...,
        [  101,  1000, 13862,  ...,     0,     0,     0],
        [  101,  1000,  2016,  ...,     0,     0,     0],
        [  101,   102,     0,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [15]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [16]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [17]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [18]:
mask_arr

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [19]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [20]:
selection[:5]

[[5, 10], [8, 10, 12, 14, 17], [], [], [7]]

In [21]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [22]:
inputs.input_ids

tensor([[  101,  1000,  1045,  ...,     0,     0,     0],
        [  101,  1000,  2647,  ..., 13529,  1012,   102],
        [  101,  1000,  2002,  ...,     0,     0,     0],
        ...,
        [  101,  1000, 13862,  ...,     0,     0,     0],
        [  101,  1000,  2016,  ...,     0,     0,     0],
        [  101,   102,     0,  ...,     0,     0,     0]])

In [23]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [24]:
dataset = MeditationsDataset(inputs)

In [25]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [26]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
#model.to(device)

In [27]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

In [28]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 2: 100%|██████████| 33/33 [01:20<00:00,  2.43s/it, loss=0.0406]


##XLM

In [29]:
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
xlmroberta_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
xlmroberta_model = XLMRobertaForMaskedLM.from_pretrained('xlm-roberta-base')

In [30]:
inputs = xlmroberta_tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [31]:
inputs

{'input_ids': tensor([[     0,     44,     87,  ...,      1,      1,      1],
        [     0,     44,  28811,  ..., 235879,      6,      2],
        [     0,     44,   1529,  ...,      1,      1,      1],
        ...,
        [     0,     44,  14838,  ...,      1,      1,      1],
        [     0,     44,   4687,  ...,      1,      1,      1],
        [     0,      2,      1,  ...,      1,      1,      1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [32]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [33]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [34]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [35]:
mask_arr

tensor([[False,  True, False,  ..., False,  True, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False,  True,  ..., False, False,  True],
        [False, False, False,  ...,  True, False, False]])

In [36]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [37]:
selection[:5]

[[1, 8, 19], [10], [3, 8], [2, 14], [9, 13, 15, 18]]

In [38]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [39]:
inputs.input_ids

tensor([[     0,    103,     87,  ...,      1,    103,      1],
        [     0,     44,  28811,  ..., 235879,      6,      2],
        [     0,     44,   1529,  ...,      1,      1,      1],
        ...,
        [     0,     44,  14838,  ...,      1,      1,      1],
        [     0,     44,    103,  ...,      1,      1,    103],
        [     0,      2,      1,  ...,    103,      1,      1]])

In [40]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [41]:
dataset = MeditationsDataset(inputs)

In [42]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [43]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
#xlmroberta_model.to(device)

In [44]:
from transformers import AdamW

# activate training mode
xlmroberta_model.train()
# initialize optimizer
optim = AdamW(xlmroberta_model.parameters(), lr=5e-5)

In [45]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # process
        outputs = xlmroberta_model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 2: 100%|██████████| 33/33 [03:33<00:00,  6.48s/it, loss=0.315]


In [46]:
bert_model = model.eval()

In [47]:
xlmroberta_model = xlmroberta_model.eval()

##BART

In [48]:
from transformers import BartTokenizer, BartForConditionalGeneration
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

In [49]:
inputs = bart_tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [50]:
inputs

{'input_ids': tensor([[    0,   113,    38,  ...,     1,     1,     1],
        [    0,   113,   796,  ..., 39058,     2,     1],
        [    0,   113,    91,  ...,     1,     1,     1],
        ...,
        [    0,   113, 24841,  ...,     1,     1,     1],
        [    0,   113,   264,  ...,     1,     1,     1],
        [    0,     2,     1,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [51]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [52]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [53]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [54]:
mask_arr

tensor([[False,  True, False,  ...,  True,  True, False],
        [False, False, False,  ..., False,  True, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ...,  True, False, False],
        [False, False, False,  ..., False, False, False]])

In [55]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [56]:
selection[:5]

[[1, 3, 7, 15, 18, 19], [4, 10, 19], [], [2, 5, 15, 18, 19], [4, 9, 14, 16]]

In [57]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [58]:
inputs.input_ids

tensor([[    0,   103,    38,  ...,   103,   103,     1],
        [    0,   113,   796,  ..., 39058,   103,     1],
        [    0,   113,    91,  ...,     1,     1,     1],
        ...,
        [    0,   113, 24841,  ...,     1,     1,     1],
        [    0,   113,   264,  ...,   103,     1,     1],
        [    0,     2,     1,  ...,     1,     1,     1]])

In [59]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [60]:
dataset = MeditationsDataset(inputs)

In [61]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [62]:
from transformers import AdamW

# activate training mode
bart_model.train()
# initialize optimizer
optim = AdamW(bart_model.parameters(), lr=5e-5)

In [63]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # process
        outputs = bart_model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 2: 100%|██████████| 33/33 [04:30<00:00,  8.20s/it, loss=0.174]


In [64]:
bart_model = bart_model.eval()

##RoBERTa

In [65]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForMaskedLM.from_pretrained('roberta-base')

In [66]:
inputs = roberta_tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [67]:
inputs

{'input_ids': tensor([[    0,   113,    38,  ...,     1,     1,     1],
        [    0,   113,   796,  ..., 39058,     2,     1],
        [    0,   113,    91,  ...,     1,     1,     1],
        ...,
        [    0,   113, 24841,  ...,     1,     1,     1],
        [    0,   113,   264,  ...,     1,     1,     1],
        [    0,     2,     1,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [68]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [69]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [70]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [71]:
mask_arr

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False,  True, False,  ..., False, False,  True],
        ...,
        [False, False, False,  ...,  True, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False,  True,  ..., False, False, False]])

In [72]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [73]:
selection[:5]

[[5, 6], [6, 7], [1, 3, 9, 20], [3, 9, 20], [12, 16, 18]]

In [74]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [75]:
inputs.input_ids

tensor([[    0,   113,    38,  ...,     1,     1,     1],
        [    0,   113,   796,  ..., 39058,     2,     1],
        [    0,   103,    91,  ...,     1,     1,   103],
        ...,
        [    0,   113, 24841,  ...,   103,     1,     1],
        [    0,   113,   264,  ...,     1,     1,     1],
        [    0,     2,   103,  ...,     1,     1,     1]])

In [76]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [77]:
dataset = MeditationsDataset(inputs)

In [78]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [79]:
from transformers import AdamW

# activate training mode
roberta_model.train()
# initialize optimizer
optim = AdamW(roberta_model.parameters(), lr=5e-5)

In [80]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # process
        outputs = roberta_model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 2: 100%|██████████| 33/33 [01:26<00:00,  2.61s/it, loss=2.38]


In [81]:
roberta_model = roberta_model.eval()

##Masked Language Modeling Pipeline

In [82]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [83]:
from transformers import XLNetTokenizer, XLNetLMHeadModel
xlnet_tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
xlnet_model = XLNetLMHeadModel.from_pretrained('xlnet-base-cased').eval()

In [84]:
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
xlmroberta_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

In [85]:
from transformers import ElectraTokenizer, ElectraForMaskedLM
electra_tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-generator')
electra_model = ElectraForMaskedLM.from_pretrained('google/electra-small-generator').eval()

In [86]:
top_k = 10

In [87]:
def decode(tokenizer, pred_idx, top_clean):
    ignore_tokens = string.punctuation + '[PAD]'
    tokens = []
    for w in pred_idx:
        token = ''.join(tokenizer.decode(w).split())
        if token not in ignore_tokens:
            tokens.append(token.replace('##', ''))
    return '\n'.join(tokens[:top_clean])

In [88]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
    text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
    # if <mask> is the last token, append a "." so that models dont predict punctuation.
    if tokenizer.mask_token == text_sentence.split()[-1]:
        text_sentence += ' .'

    input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
    mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
    return input_ids, mask_idx

In [89]:
def get_all_predictions(text_sentence, top_clean=5):
    # ========================= BERT =================================
    print(text_sentence)
    input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
    with torch.no_grad():
        predict = model(input_ids)[0]
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= XLNET LARGE =================================
    input_ids, mask_idx = encode(xlnet_tokenizer, text_sentence, False)
    perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float)
    perm_mask[:, :, mask_idx] = 1.0  # Previous tokens don't see last token
    target_mapping = torch.zeros((1, 1, input_ids.shape[1]), dtype=torch.float)  # Shape [1, 1, seq_length] => let's predict one token
    target_mapping[0, 0, mask_idx] = 1.0  # Our first (and only) prediction will be the last token of the sequence (the masked token)

    with torch.no_grad():
        predict = xlnet_model(input_ids, perm_mask=perm_mask, target_mapping=target_mapping)[0]
    xlnet = decode(xlnet_tokenizer, predict[0, 0, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= XLM ROBERTA BASE =================================
    input_ids, mask_idx = encode(xlmroberta_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = xlmroberta_model(input_ids)[0]
    xlm = decode(xlmroberta_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= BART =================================
    input_ids, mask_idx = encode(bart_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = bart_model(input_ids)[0]
    bart = decode(bart_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= ELECTRA =================================
    input_ids, mask_idx = encode(electra_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = electra_model(input_ids)[0]
    electra = decode(electra_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= ROBERTA =================================
    input_ids, mask_idx = encode(roberta_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = roberta_model(input_ids)[0]
    roberta = decode(roberta_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    return {'bert': bert,
            'xlnet': xlnet,
            'xlm': xlm,
            'bart': bart,
            'electra': electra,
            'roberta': roberta}

## after bart and roberta fine-tuning

In [90]:
text_sentence = "The wildfire <mask> through the forest at an amazing speed."
get_all_predictions(text_sentence)

The wildfire <mask> through the forest at an amazing speed.


{'bart': 'tore\nripped\nraced\nswept\nburned',
 'bert': 'ripped\nspread\nblazed\ntore\nswept',
 'electra': 'runs\npasses\nflows\nsweeps\nswept',
 'roberta': 'ripped\ntore\nswept\nraced\nmoved',
 'xlm': 'passed\nran\nspread\nruns\nwent',
 'xlnet': 'through\nwildfire\nfire\nroaring\ncircles'}

##after bart and roberta fine-tuning

In [91]:
text_sentence = "The window panes were <mask> as the wind blew through them"
get_all_predictions(text_sentence)

The window panes were <mask> as the wind blew through them


{'bart': 'cracked\nshattered\nfog\nwarped\nbent',
 'bert': 'smashed\ncracked\nshattered\ncrack\nrattled',
 'electra': 'clear\nvisible\ncovered\nclosed\nopen',
 'roberta': 'shattered\nshattering\ncollapsing\ncracking\nfalling',
 'xlm': 'open\nhigh\nempty\nhot\ncold',
 'xlnet': 'as\njust\nwere\nsuch\nopen'}

## after bart and roberta fine-tuning

In [92]:
text_sentence = "To heal and <mask> from death my heart"
get_all_predictions(text_sentence)

To heal and <mask> from death my heart


{'bart': 'guard\nprotect\nsave\nto\nrelease',
 'bert': 'preserve\nsave\nkeep\nprotect\nrelease',
 'electra': 'heal\nrecover\ndie\nlive\nsave',
 'roberta': 'recover\nprotect\nemerge\nshield\nheal',
 'xlm': 'free\nclear\nsave\nrun\nburn',
 'xlnet': 'and\nfrom\ncome\nright\naway'}

In [93]:
text_sentence ="The tax cut will <mask> the economy"
get_all_predictions(text_sentence)

The tax cut will <mask> the economy


{'bart': 'stimulate\nhurt\nfuel\nboost\nturbo',
 'bert': 'hurt\ndamage\nboost\ncrash\ncut',
 'electra': 'hurt\nhelp\nboost\nbenefit\nimprove',
 'roberta': 'boost\nstimulate\nspur\nignite\nstrengthen',
 'xlm': 'kill\nincrease\nbuild\nhurt\nfuel',
 'xlnet': 'all\nwill\nbe\nnot\nthe'}

In [94]:
text_sentence = "Black desert <mask> in iron silences"
get_all_predictions(text_sentence)

Black desert <mask> in iron silences


{'bart': 'birds\nsm\nbreat\nsw\nanimals',
 'bert': 'lay\nsat\nstood\nwaited\nreigned',
 'electra': 'is\nappears\nflies\nes\nfalls',
 'roberta': 'condensed\nmelted\nhides\nmelts\nwrapped',
 'xlm': 'ed\ns\nruns\nsit\nspread',
 'xlnet': 'and\nfor\nin\nis\non'}

##Wikipedia Tech

In [189]:
with open('stockholm/wikipedia_tech/verbs/wiki_tech_verbs_masked.txt', 'r') as fp:
    text = fp.read().split('\n')

In [190]:
text

['\'[\'Technology ("science of craft", from Greek τέχνη, techne, "art, skill, cunning of hand"; and -λογία, -logia) is the sum of techniques, skills, methods, and processes  <mask> in the production of goods or services or in the accomplishment of objectives, such as scientific investigation\',',
 "'Technology  <mask> be the knowledge of techniques, processes, and the like, or it  ',",
 "'<mask> be  <mask> in machines to  <mask> for operation without detailed knowledge of their workings Systems (eg machines)  <mask> technology by  <mask> an input,  <mask> it  ',",
 "'<mask> to the system\\'s use, and then  <mask> an outcome',",
 "'are  <mask> to as technology systems or technological systems\\nThe simplest form of technology is the development and use of basic tools',",
 "'The prehistoric invention of shaped stone tools  <mask> by the discovery of how to  <mask> fire  <',",
 "'mask> sources of food',",
 "'The later Neolithic Revolution  <mask> this, and  <mask> the sustenance available

In [191]:
text_sentence = "Technology (science of craft, from Greek τέχνη, techne, art, skill, cunning of hand; and -λογία, -logia) is the sum of techniques, skills, methods, and processes <mask> in the production of goods or services or in the accomplishment of objectives, such as scientific investigation"
get_all_predictions(text_sentence)

Technology (science of craft, from Greek τέχνη, techne, art, skill, cunning of hand; and -λογία, -logia) is the sum of techniques, skills, methods, and processes <mask> in the production of goods or services or in the accomplishment of objectives, such as scientific investigation


{'bart': 'used\nthat\nemployed\ndeveloped\nneeded',
 'bert': 'used\nemployed\napplied\nutilized\ninvolved',
 'electra': 'involved\nemployed\nused\nengaged\nutilized',
 'roberta': 'used\nemployed\ninvolved\nutilized\ninstrumental',
 'xlm': 'involved\nused\nassociated\ninherent\noperating',
 'xlnet': 'used\nemployed\ninvolved\napplied\nuseful'}

In [192]:
text_sentence = " machines) <mask> technology by taking an input, changing it according to the system\'s use, and then producing an outcome are referred to as technology systems or technological systems"
get_all_predictions(text_sentence)

 machines) <mask> technology by taking an input, changing it according to the system's use, and then producing an outcome are referred to as technology systems or technological systems


{'bart': 'that\nwhich\ndesigned\nThat\ndevelop',
 'bert': 'process\ndeveloping\napplying\ngenerating\nadvancing',
 'electra': 'and\nof\nor\nusing\ngenerating',
 'roberta': 'advancing\naltering\nevolving\nbuilding\nsustaining',
 'xlm': 'build\ninvent\ndesign\nrepair\nmanufacture',
 'xlnet': 'or\nand\nusing\nin\nmeans'}

In [193]:
text_sentence = "The prehistoric invention of shaped stone tools <mask> by the discovery of how to control fire increased sources of food"
get_all_predictions(text_sentence)

The prehistoric invention of shaped stone tools <mask> by the discovery of how to control fire increased sources of food


{'bart': 'spurred\nwas\nfueled\nfollowed\naccompanied',
 'bert': 'followed\naided\nfueled\naccompanied\nspurred',
 'electra': 'inspired\nfollowed\ncaused\ncreated\nled',
 'roberta': 'followed\naccompanied\nspurred\nbolstered\npreceded',
 'xlm': 'followed\nled\ninspired\ncaused\nhelped',
 'xlnet': 'made\nby\nused\nreplaced\nintroduced'}

In [194]:
text_sentence = "The invention of the wheel <mask> humans to travel in and control their environment"
get_all_predictions(text_sentence)

The invention of the wheel <mask> humans to travel in and control their environment


{'bart': 'enabled\nallowed\nempowered\nfreed\nmade',
 'bert': 'allowed\nenabled\npermitted\ntaught\nencouraged',
 'electra': 'allows\nenables\nrequires\nallowed\npermits',
 'roberta': 'enabled\nallowed\nempowered\nallows\npermitted',
 'xlm': 'allowed\nenabled\nallows\nhelped\ntold',
 'xlnet': 'human\nman\nhumans\nfor\nin'}

In [195]:
text_sentence = "', 'Seagate Technology Holdings plc, an Irish public limited company (commonly <mask> to as Seagate), is an American data storage company"
get_all_predictions(text_sentence)

', 'Seagate Technology Holdings plc, an Irish public limited company (commonly <mask> to as Seagate), is an American data storage company


{'bart': 'referred\nrefers\nrefer\nreferring\nabbrevi',
 'bert': 'referred\nrefers\nrefer\nshortened\nreferring',
 'electra': 'referred\nknown\nrefer\nabbreviated\nspoken',
 'roberta': 'referred\nReferred\nrefers\nreferring\nshortened',
 'xlm': 'refer\nthought\nknown\nreported\ndescribed',
 'xlnet': 'ly\nreferred\nsimply\ncommonly\nknown'}

In [196]:
text_sentence = "It was <mask> in 1978 as Shugart Technology and commenced business in 1979"
get_all_predictions(text_sentence)

It was <mask> in 1978 as Shugart Technology and commenced business in 1979


{'bart': 'founded\nincorporated\nformed\noriginally\nstarted',
 'bert': 'founded\nestablished\nincorporated\nformed\nstarted',
 'electra': 'rebranded\nacquired\nknown\nestablished\nrenamed',
 'roberta': 'incorporated\nfounded\nestablished\nformed\nregistered',
 'xlm': 'established\nborn\nbuilt\ncreated\nfound',
 'xlnet': 'in\nestablished\nalso\nthen\nwas'}

In [197]:
text_sentence = " Since 2010, the company has been <mask> in Dublin, Ireland, with operational headquarters in Fremont, California, United States"
get_all_predictions(text_sentence)

 Since 2010, the company has been <mask> in Dublin, Ireland, with operational headquarters in Fremont, California, United States


{'bart': 'headquartered\nbased\nlocated\nincorporated\nhoused',
 'bert': 'based\nheadquartered\nlocated\noperating\ncentered',
 'electra': 'headquartered\noperating\nbased\nlocated\noperational',
 'roberta': 'headquartered\nbased\nlocated\noperating\nregistered',
 'xlm': 'operating\nestablished\nbased\nlocated\nbuilt',
 'xlnet': 'in\nheadquartered\nbased\nlocated\noperating'}

In [198]:
text_sentence = "', 'A technology company (or tech company) is an electronics-based technology company, <mask>, for example, business relating to digital electronics, software, and internet-related services, such as e-commerce services"
get_all_predictions(text_sentence)

', 'A technology company (or tech company) is an electronics-based technology company, <mask>, for example, business relating to digital electronics, software, and internet-related services, such as e-commerce services


{'bart': 'for\nwith\noften\nbased\n</s>',
 'bert': 'offering\nproviding\ncomprising\nwith\nhandling',
 'electra': 'offering\nor\nproviding\nincluding\nin',
 'roberta': 'meaning\nwith\nconcentrating\nincluding\ntargeting',
 'xlm': 'operating\noffering\nwhich\nrunning\nthat',
 'xlnet': 'with\nhaving\nand\nincluding\nor'}

In [199]:
text_sentence = "', 'An institute of technology (also referred to as: technological university, technical university, university of technology, technological educational institute, technical college, polytechnic university or just polytechnic) is an institution of tertiary education (such as a university or college) that <mask> in engineering, technology, applied science, and natural sciences"
get_all_predictions(text_sentence)

', 'An institute of technology (also referred to as: technological university, technical university, university of technology, technological educational institute, technical college, polytechnic university or just polytechnic) is an institution of tertiary education (such as a university or college) that <mask> in engineering, technology, applied science, and natural sciences


{'bart': 'specializes\ntrains\nteaches\nspecial\nfost',
 'bert': 'specializes\nfocuses\nworks\nspecialized\nengages',
 'electra': 'specializes\nmajors\nspecialized\nworks\nengages',
 'roberta': 'specializes\nspecialized\ndeals\nadvances\nworks',
 'xlm': 'works\ncourses\nruns\nprojects\ncentres',
 'xlnet': 'specializes\nis\nspecialized\nresults\ndeals'}

In [200]:
text_sentence = " The term technology <mask> from the Greek word techne, meaning art and craft, and the word logos, meaning word and speech"
get_all_predictions(text_sentence)

 The term technology <mask> from the Greek word techne, meaning art and craft, and the word logos, meaning word and speech


{'bart': 'derives\ncomes\nstems\nis\nderived',
 'bert': 'derives\ncomes\noriginates\noriginated\nderived',
 'electra': 'derives\ncomes\noriginates\nderived\ndiffers',
 'roberta': 'derives\nstems\ncomes\nevolved\ncame',
 'xlm': 'comes\ncame\nbenefits\nruns\nstem',
 'xlnet': 'comes\nderives\nstems\ncame\nis'}

In [202]:
text_sentence = "'Information technology (IT) <mask> the use of computers to create, process, store and exchange all kinds of electronic data and information"
get_all_predictions(text_sentence)

'Information technology (IT) <mask> the use of computers to create, process, store and exchange all kinds of electronic data and information


{'bart': 'refers\nis\ninvolves\nhas\nencompasses',
 'bert': 'is\ninvolves\nencompasses\nincludes\ndescribes',
 'electra': 'is\ninvolves\nrequires\npromotes\nsupports',
 'roberta': 'facilitates\nstimulates\nenables\nmeans\nencourages',
 'xlm': 'includes\nis\nmeans\nallows\nmakes',
 'xlnet': 'is\nthe\nit\nto\nor'}

In [203]:
text_sentence = "IT is typically used within the context of business operations as <mask> to personal or entertainment technologies"
get_all_predictions(text_sentence)

IT is typically used within the context of business operations as <mask> to personal or entertainment technologies


{'bart': 'opposed\ncompared\nan\na\nit',
 'bert': 'opposed\napplied\ncompared\nrelated\nrelating',
 'electra': 'opposed\ncompared\nregards\nalternatives\nrelated',
 'roberta': 'opposed\ncompared\nalternatives\nadjunct\nkey',
 'xlm': 'compared\nalternative\nwell\naddition\nsimilar',
 'xlnet': 'as\nto\nAs\na\nnot'}

In [204]:
text_sentence = "IT is <mask> to be a subset of information and communications technology (ICT)"
get_all_predictions(text_sentence)

IT is <mask> to be a subset of information and communications technology (ICT)


{'bart': 'often\nconsidered\nnow\nbelieved\nperceived',
 'bert': 'considered\nthought\nintended\nsupposed\nproposed',
 'electra': 'considered\nthought\nbelieved\nintended\nsaid',
 'roberta': 'considered\nunderstood\nthought\nbelieved\nsaid',
 'xlm': 'thought\nsupposed\nconsidered\nunderstood\nsaid',
 'xlnet': 'is\nto\nnot\nI\nare'}

In [205]:
text_sentence = " An information technology system (IT system) is generally an information system, a communications system, or, more specifically <mask>, a computer system – including all hardware, software, and peripheral equipment – operated by a limited group of IT users"
get_all_predictions(text_sentence)

 An information technology system (IT system) is generally an information system, a communications system, or, more specifically <mask>, a computer system – including all hardware, software, and peripheral equipment – operated by a limited group of IT users


{'bart': 'computer\n</s>\nPC\nbusiness\n",',
 'bert': 'informally\ncolloquially\nspecifically\nbroadly\nproperly',
 'electra': 'to\nsimply\nspeaking\nnot\ncomputing',
 'roberta': 'speaking\nput\nnow\ntoday\never',
 'xlm': '</s>\nthat\nyet\ntold\nly',
 'xlnet': 'generally\na\ntypically\nherein\nan'}

In [206]:
text_sentence = "Humans have been <mask>, retrieving, manipulating, and communicating information since the Sumerians in Mesopotamia developed writing in about 3000 BC"
get_all_predictions(text_sentence)

Humans have been <mask>, retrieving, manipulating, and communicating information since the Sumerians in Mesopotamia developed writing in about 3000 BC


{'bart': 'writing\nencoding\nlearning\nmanipulating\ncreating',
 'bert': 'collecting\nrecording\ngathering\nacquiring\nmanipulating',
 'electra': 'collecting\nmanipulating\nanalyzing\ncreating\ndeveloping',
 'roberta': 'storing\ngathering\naccessing\nprocessing\ncollecting',
 'xlm': 'building\ncreating\nworking\noperating\nsitting',
 'xlnet': 'making\ncollecting\nplaying\nbeing\naround'}

In [207]:
text_sentence = "We shall <mask> it information technology (IT)"
get_all_predictions(text_sentence)

We shall <mask> it information technology (IT)


{'bart': 'call\nnow\nrefer\nrename\nlabel',
 'bert': 'call\nname\nconsider\nterm\nteach',
 'electra': 'use\ncall\nhave\nconsider\nprovide',
 'roberta': 'call\ndub\nname\nlabel\nrename',
 'xlm': 'make\npass\nfund\nrepair\ntreat',
 'xlnet': 'be\nhave\nis\ncall\ncalled'}

In [208]:
text_sentence = "The term is commonly used as a synonym for computers and computer networks, but it also <mask> other information distribution technologies such as television and telephones"
get_all_predictions(text_sentence)

The term is commonly used as a synonym for computers and computer networks, but it also <mask> other information distribution technologies such as television and telephones


{'bart': 'refers\napplies\nreflects\ndenotes\ndescribes',
 'bert': 'encompasses\ndescribes\nincludes\nincorporates\ndenotes',
 'electra': 'describes\nuses\nincludes\nmeans\ncovers',
 'roberta': 'encompasses\ncovers\nincludes\nexcludes\nmeans',
 'xlm': 'includes\nruns\nbenefits\nmeans\nhas',
 'xlnet': 'for\nincludes\nencompasses\nterms\ncovers'}

In [209]:
text_sentence = "Based on the storage and processing technologies <mask>, it is possible to distinguish four distinct phases of IT development: pre-mechanical (3000 BC – 1450 AD), mechanical (1450–1840), electromechanical (1840–1940), and electronic (1940–present)"
get_all_predictions(text_sentence)

Based on the storage and processing technologies <mask>, it is possible to distinguish four distinct phases of IT development: pre-mechanical (3000 BC – 1450 AD), mechanical (1450–1840), electromechanical (1840–1940), and electronic (1940–present)


{'bart': 'used\ndeveloped\nthat\ninvented\nin',
 'bert': 'used\navailable\nemployed\ninvolved\nstudied',
 'electra': 'described\npresented\nshown\ndiscussed\ndisclosed',
 'roberta': 'used\npresent\navailable\ndeveloped\ntested',
 'xlm': 'developed\nexisting\nassociated\nconcerned\n</s>',
 'xlnet': 'developed\nused\nmentioned\navailable\ndescribed'}

In [210]:
text_sentence = "This article <mask> on the most recent period (electronic)"
get_all_predictions(text_sentence)

This article <mask> on the most recent period (electronic)


{'bart': 'is\nreflects\nwas\nfocuses\ncenters',
 'bert': 'focuses\nfocused\nreflects\nfocus\nis',
 'electra': 'focuses\nfocused\ntouches\nreflects\ntouched',
 'roberta': 'focuses\nhinges\nfocused\ntouches\ndepends',
 'xlm': 'reports\nruns\nappeared\nfocused\nreported',
 'xlnet': 'on\ntoday\nI\narticle\nthe'}

In [211]:
text_sentence = " When <mask> to with its abbreviation, EdTech, it is often referring to the industry of companies that create educational technology"
get_all_predictions(text_sentence)

 When <mask> to with its abbreviation, EdTech, it is often referring to the industry of companies that create educational technology


{'bart': 'the\nsomeone\nthis\nreferring\nreferred',
 'bert': 'referred\nrefers\nrefer\nreferring\nrelated',
 'electra': 'compared\nreferring\nreferred\napplied\nsticking',
 'roberta': 'referred\nspoken\nReferred\ncoming\npointed',
 'xlm': 'spoke\ncoming\ngetting\nmade\ncompared',
 'xlnet': 'referring\nWhen\nit\ncomes\ncoming'}

In [212]:
text_sentence = "Massachusetts Institute of Technology (MIT) <mask> a private land-grant research university in Cambridge, Massachusetts"
get_all_predictions(text_sentence)

Massachusetts Institute of Technology (MIT) <mask> a private land-grant research university in Cambridge, Massachusetts


{'bart': 'is\nwas\nbegan\nrefers\noperates',
 'bert': 'is\nwas\noperates\nremains\nmaintains',
 'electra': 'is\nwas\noperates\nhas\nare',
 'roberta': 'is\noperates\nmanages\nestablished\nruns',
 'xlm': 'is\nwas\nhas\nas\nprovides',
 'xlnet': 'is\nI\na\nwas\nare'}

In [213]:
text_sentence = "The institute has an urban campus that <mask> more than a mile (1"
get_all_predictions(text_sentence)

The institute has an urban campus that <mask> more than a mile (1


{'bart': 'extends\nis\nmeasures\nstretches\nspans',
 'bert': 'extends\nstretches\nspans\nis\nencompasses',
 'electra': 'runs\nspans\nextends\nis\nstretches',
 'roberta': 'stretches\nextends\nspans\nstretched\nspreads',
 'xlm': 'is\nruns\nwas\nhas\ncosts',
 'xlnet': 'is\nextends\nthat\nmore\nstands'}

In [214]:
text_sentence = "6 km) alongside the Charles River, and <mask> a number of major off-campus facilities such as the MIT Lincoln Laboratory, the Bates Center, and the Haystack Observatory, as well as affiliated laboratories such as the Broad and Whitehead Institutes"
get_all_predictions(text_sentence)

6 km) alongside the Charles River, and <mask> a number of major off-campus facilities such as the MIT Lincoln Laboratory, the Bates Center, and the Haystack Observatory, as well as affiliated laboratories such as the Broad and Whitehead Institutes


{'bart': 'host\n."\ncontain\nconnect\nthe',
 'bert': 'with\nincluding\ncomprising\ncontaining\nadjoining',
 'electra': 'with\nthrough\nin\nat\nto',
 'roberta': 'to\nwith\nover\nlinking\nalso',
 'xlm': 'has\nincludes\nprovides\nalong\nruns',
 'xlnet': 'includes\nat\ninclude\nin\nalso'}

In [215]:
text_sentence = " As of June 2021, 97 Nobel laureates, 26 Turing Award winners, and 8 Fields Medalists have been <mask> with MIT as alumni, faculty members, or researchers"
get_all_predictions(text_sentence)

 As of June 2021, 97 Nobel laureates, 26 Turing Award winners, and 8 Fields Medalists have been <mask> with MIT as alumni, faculty members, or researchers


{'bart': 'affiliated\nassociated\nhonored\nconnected\npermanently',
 'bert': 'associated\naffiliated\nlinked\nconnected\nbestowed',
 'electra': 'registered\nassociated\nlisted\naffiliated\nidentified',
 'roberta': 'registered\nassociated\nconnected\naffiliated\nlinked',
 'xlm': 'associated\nregistered\nidentified\nestablished\ntrading',
 'xlnet': 'associated\nidentified\nbeen\nliving\nlisted'}

In [216]:
text_sentence = " In addition, 58 National Medal of Science recipients, 29 National Medals of Technology and Innovation recipients, 50 MacArthur Fellows, 80 Marshall Scholars, 3 Mitchell Scholars, 22 Schwarzman Scholars, 9 Knight-Hennessy Scholars,  41 astronauts, and 16 Chief Scientists of the U, Air Force have been <mask> with MIT"
get_all_predictions(text_sentence)

 In addition, 58 National Medal of Science recipients, 29 National Medals of Technology and Innovation recipients, 50 MacArthur Fellows, 80 Marshall Scholars, 3 Mitchell Scholars, 22 Schwarzman Scholars, 9 Knight-Hennessy Scholars,  41 astronauts, and 16 Chief Scientists of the U, Air Force have been <mask> with MIT


{'bart': 'affiliated\nhonored\nassociated\nawarded\ntied',
 'bert': 'affiliated\nassociated\nconnected\nlinked\ninvolved',
 'electra': 'working\naffiliated\ncollaborating\ninvolved\nregistered',
 'roberta': 'affiliated\nconnected\nconferred\nassociated\nlinked',
 'xlm': 'associated\nworking\ntrading\nregistered\nalong',
 'xlnet': 'associated\nconferred\naffiliated\nawarded\nhonored'}

In [217]:
text_sentence = "MIT <mask> a member of the Association of American Universities (AAU)"
get_all_predictions(text_sentence)

MIT <mask> a member of the Association of American Universities (AAU)


{'bart': 'is\nwas\nbecame\nhas\nIS',
 'bert': 'is\nwas\nbecame\nremains\nare',
 'electra': 'is\nwas\nand\nas\nbecame',
 'roberta': 'is\nwas\nare\nwere\nbecoming',
 'xlm': 'is\nwas\nas\nbecame\nbeing',
 'xlnet': 'MIT\nis\na\nare\ncampus'}

In [218]:
text_sentence = "Gigabyte Technology (<mask> as GIGABYTE or sometimes GIGA-BYTE; formally GIGA-BYTE Technology Co"
get_all_predictions(text_sentence)

Gigabyte Technology (<mask> as GIGABYTE or sometimes GIGA-BYTE; formally GIGA-BYTE Technology Co


{'bart': 'commonly\noften\nalso\nknown\nformally',
 'bert': 'abbreviated\nknown\nstyled\ninformally\ncolloquially',
 'electra': 'such\nknown\ntranslated\nalso\nlatin',
 'roberta': 'traded\nknown\ntrading\nknown\nReferred',
 'xlm': 'seen\nlanguage\nbased\nmarked\nknown',
 'xlnet': 'sometimes\nsometimes\nor\noften'}

In [219]:
text_sentence = ") <mask> a Taiwanese manufacturer and distributor of computer hardware"
get_all_predictions(text_sentence)

) <mask> a Taiwanese manufacturer and distributor of computer hardware


{'bart': 'Taiwan\nPCB\nLenovo\nHTC\nTos',
 'bert': 'is\nwas\nbeing\n•\ndenotes',
 'electra': 'is\nas\n–',
 'roberta': 'Moor\nAfterwards\nbargaining\n<pad>\nand',
 'xlm': '</s>\nhaft\nlook\n")\nthrough',
 'xlnet': 'is\nas\nor\n<eop>\na'}

In [220]:
text_sentence = "8 million motherboards in the first quarter of 2015, which <mask> it to become the leading motherboard vendor"
get_all_predictions(text_sentence)

8 million motherboards in the first quarter of 2015, which <mask> it to become the leading motherboard vendor


{'bart': 'propelled\nenabled\ncatapult\nallowed\nhelped',
 'bert': 'enabled\nallowed\nhelped\nenables\nled',
 'electra': 'led\nallowed\nenabled\ncaused\nforced',
 'roberta': 'enabled\npropelled\nallowed\nled\npushed',
 'xlm': 'allowed\nallows\ncaused\nled\nenabled',
 'xlnet': 'it\nallowed\nmade\ngave\nled'}

In [222]:
text_sentence = " Gigabyte also <mask> custom graphics cards and laptop computers (including thin and light laptops under its Aero sub-brand)"
get_all_predictions(text_sentence)

 Gigabyte also <mask> custom graphics cards and laptop computers (including thin and light laptops under its Aero sub-brand)


{'bart': 'makes\nmanufactures\ndesigned\nproduces\nbuilds',
 'bert': 'produced\nproduces\nmanufactures\nmanufactured\nsold',
 'electra': 'manufactures\nsells\nincludes\nmakes\nfeatures',
 'roberta': 'manufactures\nsells\nproduces\nmakes\nbuilds',
 'xlm': 'makes\noffers\nprovides\nbuilt\nruns',
 'xlnet': 'also\nproduces\nsupports\nmakes\nsells'}

In [224]:
text_sentence = " In 2010, Gigabyte was <mask> 17th in Taiwan's Top 20 Global Brands by the Taiwan External Trade Development Council"
get_all_predictions(text_sentence)

 In 2010, Gigabyte was <mask> 17th in Taiwan's Top 20 Global Brands by the Taiwan External Trade Development Council


{'bart': 'ranked\nplaced\nlisted\nnamed\nrated',
 'bert': 'ranked\nlisted\nplaced\nrated\nranking',
 'electra': 'ranked\nplaced\nnamed\nselected\nlisted',
 'roberta': 'ranked\nplaced\nrated\nnamed\nvoted',
 'xlm': 'listed\nmade\nplaced\nfound\ntagged',
 'xlnet': 'ranked\nnamed\nvoted\nlisted\nthe'}

In [225]:
text_sentence = "The company is publicly held and <mask> on the Taiwan Stock Exchange, stock ID number TWSE: 2376"
get_all_predictions(text_sentence)

The company is publicly held and <mask> on the Taiwan Stock Exchange, stock ID number TWSE: 2376


{'bart': 'traded\nlisted\ntrades\nis\ntrading',
 'bert': 'traded\nlisted\nfloated\ntrading\ntrades',
 'electra': 'listed\noperated\ntraded\nregistered\nsold',
 'roberta': 'traded\nlisted\ntrades\nregistered\nfloated',
 'xlm': 'registered\ntrading\nsitting\nlisted\noperating',
 'xlnet': 'and\non\nnot\nheld\nsilent'}

In [226]:
text_sentence = "The Indian Institutes of Technology (IITs) are autonomous public technical universities <mask> across India"
get_all_predictions(text_sentence)

The Indian Institutes of Technology (IITs) are autonomous public technical universities <mask> across India


{'bart': 'located\noperating\nspread\nthat\nbased',
 'bert': 'spread\nscattered\nlocated\ndistributed\nsituated',
 'electra': 'all\nlocated\nfrom\noperating\ndistributed',
 'roberta': 'spread\nscattered\nall\nlocated\ndotted',
 'xlm': 'from\noperating\nlocated\nspread\nrunning',
 'xlnet': 'scattered\nspread\nlocated\nin\nsituated'}

In [227]:
text_sentence = " The Institutes of Technology Act, 1961 <mask> twenty-three institutes"
get_all_predictions(text_sentence)

 The Institutes of Technology Act, 1961 <mask> twenty-three institutes


{'bart': 'created\nestablished\nempowered\nvested\nincorporated',
 'bert': 'established\ncreated\nlisted\nadded\nrecognized',
 'electra': 'created\nauthorized\nincludes\napproved\nhas',
 'roberta': 'authorized\nestablished\ncreated\ndesignated\nnamed',
 'xlm': 'includes\nprovides\ncovered\nbenefits\nestablished',
 'xlnet': 'twenty\nsixty\nthirty\nforty\nseventy'}

In [228]:
text_sentence = "The Minister of Education <mask> the ex officio Chairperson of the IIT Council"
get_all_predictions(text_sentence)

The Minister of Education <mask> the ex officio Chairperson of the IIT Council


{'bart': 'was\nis\nbecame\nappointed\nbecomes',
 'bert': 'is\nwas\nand\nbecame\nbecomes',
 'electra': 'is\nwas\nand\nbecame\nas',
 'roberta': 'is\nwas\nsummoned\nand\nmet',
 'xlm': 'was\nis\nbecame\nand\nas',
 'xlnet': 'is\nthe\nwas\nand\nas'}

In [229]:
text_sentence = "As of 2020, the total number of seats for undergraduate programs in all IITs <mask> 16,053"
get_all_predictions(text_sentence)

As of 2020, the total number of seats for undergraduate programs in all IITs <mask> 16,053


{'bart': 'exceeded\nwas\nsurpassed\nexceeds\ntotaled',
 'bert': 'is\nwas\nexceeds\nexceed\nexceeded',
 'electra': 'is\nwas\nof\nat\nare',
 'roberta': 'was\nis\ntotaled\nwere\nexceeds',
 'xlm': 'is\nwas\nwere\nat\nare',
 'xlnet': 's\nis\nat\nwas\nare'}

In [230]:
text_sentence = " is an American producer of computer memory and computer data storage <mask> dynamic random-access memory, flash memory, and USB flash drives"
get_all_predictions(text_sentence)

 is an American producer of computer memory and computer data storage <mask> dynamic random-access memory, flash memory, and USB flash drives


{'bart': 'devices\nequipment\nproducts\nsystems\nappliances',
 'bert': 'including\nwith\nusing\nvia\nfor',
 'electra': 'including\nwith\nfor\nusing\nin',
 'roberta': 'with\nthrough\nincluding\nusing\nutilizing',
 'xlm': 'through\nwith\nin\nusing\nby',
 'xlnet': 'including\nlike\nespecially\nsuch\nand'}

In [231]:
text_sentence = " It <mask> headquartered in Boise, Idaho"
get_all_predictions(text_sentence)

 It <mask> headquartered in Boise, Idaho


{'bart': 'is\nwas\nhas\nbeing\ncurrently',
 'bert': 'is\nwas\nremains\ncurrently\nhas',
 'electra': 'is\nwas\nremains\nare\nlies',
 'roberta': "is\nwas\nare\nwere\n's",
 'xlm': 'was\nis\ns\nwere\nhas',
 'xlnet': 'is\nwas\nlocated\nalso\ncurrently'}

In [232]:
text_sentence = "Its consumer products are <mask> under the brands Crucial  and Ballistix"
get_all_predictions(text_sentence)

Its consumer products are <mask> under the brands Crucial  and Ballistix


{'bart': 'sold\nmarketed\nmanufactured\ndistributed\npackaged',
 'bert': 'sold\nmarketed\ndistributed\navailable\nproduced',
 'electra': 'available\nmarketed\nmanufactured\nsold\nproduced',
 'roberta': 'sold\ngrouped\nmarketed\nbundled\ntraded',
 'xlm': 'covered\nlisted\nrun\nsplit\nlocated',
 'xlnet': 'are\nthe\ndirected\nrespectively\nis'}

In [233]:
text_sentence = " Micron and Intel together <mask> IM Flash Technologies, which produces NAND flash memory"
get_all_predictions(text_sentence)

 Micron and Intel together <mask> IM Flash Technologies, which produces NAND flash memory


{'bart': 'control\nacquired\nfounded\nformed\nowns',
 'bert': 'created\ndeveloped\nfounded\ncreate\nform',
 'electra': 'develop\nform\nhave\ndeveloped\nuse',
 'roberta': 'formed\nform\ncreated\nfounded\nfound',
 'xlm': 'built\ncreated\ndeveloped\nproduce\nestablished',
 'xlnet': 'with\nas\ntogether\nto\nin'}

In [234]:
text_sentence = "It <mask> Lexar between 2006 and 2017"
get_all_predictions(text_sentence)

It <mask> Lexar between 2006 and 2017


{'bart': 'was\ncompeted\nraced\ntraded\nreplaced',
 'bert': 'hosted\nmanufactured\nproduced\nwas\nsupplied',
 'electra': 'became\nwas\nreplaced\nsold\nhad',
 'roberta': 'drove\ntraded\nmoved\nused\nmaintained',
 'xlm': 'won\nbought\narrived\nbuilt\nserved',
 'xlnet': 'was\nin\nbecame\nIt\nit'}

In [138]:
text_sentence = "It was originally  <mask> in 1929 as american record corporation"
get_all_predictions(text_sentence)

It was originally  <mask> in 1929 as american record corporation


{'bart': 'released\nissued\npublished\nrecorded\nproduced',
 'bert': 'established\nfounded\nformed\nlaunched\nincorporated',
 'electra': 'established\nincorporated\nknown\nfounded\nformed',
 'roberta': 'formed\nestablished\nfounded\nregistered\nconceived',
 'xlm': 'established\ncreated\nbuilt\nopened\nbought',
 'xlnet': 'in\nlater\noriginally\nfirst\nreleased'}

In [139]:
text_sentence = "The merger was  <mask> on August 1, 2019"
get_all_predictions(text_sentence)

The merger was  <mask> on August 1, 2019


{'bart': 'finalized\neffective\napproved\nformally\nofficially',
 'bert': 'completed\nfinalized\nannounced\napproved\ncomplete',
 'electra': 'announced\nlaunched\nconcluded\ncompleted\nfinalized',
 'roberta': 'finalized\ncompleted\nsealed\nsigned\nclosed',
 'xlm': 'announced\nreported\nplanned\ndecided\npassed',
 'xlnet': 'was\non\nagain\nthen\nmade'}

In [140]:
text_sentence = "Country music often  <mask> of ballads and dance tunes with generally simple forms, folk lyrics, and harmonies "
get_all_predictions(text_sentence)

Country music often  <mask> of ballads and dance tunes with generally simple forms, folk lyrics, and harmonies 


{'bart': 'consists\nconsisted\nconsist\nincorporates\ncontains',
 'bert': 'consisted\nconsists\nconsist\nconsisting\ncomposed',
 'electra': 'consists\nspeaks\nconsist\nconsisted\ntalks',
 'roberta': 'consists\nconsisted\ncomprises\ncomposed\nconsist',
 'xlm': 'consist\nmade\ncompose\nheard\nmeans',
 'xlnet': 'often\nconsists\nprimarily\nmostly\nconsisting'}

In [141]:
text_sentence = "It  <mask> more than three million album entries and 30 million tracks"
get_all_predictions(text_sentence)

It  <mask> more than three million album entries and 30 million tracks


{'bart': 'sold\nhas\nhad\namassed\nspawned',
 'bert': 'had\naccumulated\nlists\nhas\nfeatures',
 'electra': 'has\nhad\nincluded\nincludes\ncontains',
 'roberta': 'amassed\nhad\nattracted\ngenerated\naccumulated',
 'xlm': 'has\nhad\nmade\nproduced\nreceived',
 'xlnet': 'more\nhas\nis\nmakes\nthem'}

In [142]:
text_sentence = "<mask> in 1991, the database was first made available on the Internet in 1994'"
get_all_predictions(text_sentence)

<mask> in 1991, the database was first made available on the Internet in 1994'


{'bart': '<s>\n"\'\n</s>\n""\n"[',
 'bert': 'completed\nbeginning\ncreated\nestablished\nbegun',
 'electra': 'and\nbuilt\nback\nfounded\nstarting',
 'roberta': 'Met\ninitiated\nbeginning\nreed\nFed',
 'xlm': '</s>\nMeet\nThrough\nຮຽນພາສາອັງກິດ\nפרייטאג',
 'xlnet': 'in\nagain\ncopyright\nrenewed\ned'}

In [143]:
text_sentence = "AllMusic is  <mask> by RhythmOne"
get_all_predictions(text_sentence)

AllMusic is  <mask> by RhythmOne


{'bart': 'distributed\nendorsed\npublished\nowned\naccredited',
 'bert': 'compiled\nprovided\nproduced\nrun\npresented',
 'electra': 'produced\ndirected\nwritten\nedited\nreviewed',
 'roberta': 'administered\nsupplied\nsponsored\nlicensed\ncomposed',
 'xlm': 'provided\noffered\nrun\nproduced\ncovered',
 'xlnet': 'is\nwas\nmade\nprovided\ndone'}

In [144]:
text_sentence = "Music! Music! Music! is a popular song  <mask> by Stephen Weiss and Bernie Baum "
get_all_predictions(text_sentence)

Music! Music! Music! is a popular song  <mask> by Stephen Weiss and Bernie Baum 


{'bart': 'written\ncomposed\npenned\nrecorded\nco',
 'bert': 'written\ncomposed\nrecorded\nsung\npenned',
 'electra': 'written\ncomposed\nsung\nperformed\ninspired',
 'roberta': 'composed\nwritten\ndeveloped\npenned\nsung',
 'xlm': 'written\ninspired\nsung\nproduced\nmade',
 'xlnet': 'by\ndirected\nservice\nstarted\nplan'}

In [145]:
text_sentence = "EDM producers also  <mask> their music live in a concert"
get_all_predictions(text_sentence)

EDM producers also  <mask> their music live in a concert


{'bart': 'play\nperform\nbroadcast\nlike\nstream',
 'bert': 'perform\nperformed\nstreamed\nperforms\nplayed',
 'electra': 'play\nrecorded\nput\nperform\nplayed',
 'roberta': 'played\nperformed\nplay\ndelivered\nsang',
 'xlm': 'make\nput\nmade\nturn\nhad',
 'xlnet': 'also\ntheir\nmade\nmake\nrecorded'}

In [146]:
text_sentence = "UMG\'s corporate headquarters are <mask> in Hilversum, Netherlands"
get_all_predictions(text_sentence)

UMG's corporate headquarters are <mask> in Hilversum, Netherlands


{'bart': 'located\nbased\nnow\nsituated\ncurrently',
 'bert': 'located\nbased\nsituated\nheadquartered\nestablished',
 'electra': 'located\nbased\nheadquartered\nsituated\ncentered',
 'roberta': 'located\nsituated\nnow\nbased\nalso',
 'xlm': 'located\nbuilt\nestablished\nright\nsitting',
 'xlnet': 'in\nare\nat\nlocated\nbased'}

In [147]:
text_sentence = "UMG has  <mask> licensing agreements with more than 400 platforms worldwide"
get_all_predictions(text_sentence)

UMG has  <mask> licensing agreements with more than 400 platforms worldwide


{'bart': 'entered\nsigned\nnumerous\nlicensing\nforged',
 'bert': 'signed\nlicensed\nestablished\nnegotiated\nlicensing',
 'electra': 'signed\ndeveloped\nreached\nestablished\nhad',
 'roberta': 'signed\nentered\nsecured\nnegotiated\nexclusive',
 'xlm': 'signed\nreached\nhad\nwritten\nbuilt',
 'xlnet': 'has\nhad\nhave\ninternational\nglobal'}

In [148]:
text_sentence = "Gospel music is a genre of Christian music, Gospel music can be <mask> to the early 17th century"
get_all_predictions(text_sentence)

Gospel music is a genre of Christian music, Gospel music can be <mask> to the early 17th century


{'bart': 'traced\ndated\nattributed\ntrace\ncredited',
 'bert': 'traced\ndated\ndate\ndates\ntrace',
 'electra': 'traced\ndated\nattributed\nused\nadapted',
 'roberta': 'traced\ndated\nattributed\nback\ntrace',
 'xlm': 'dating\nback\ndates\nrelated\ntied',
 'xlnet': 'be\nback\nfound\ncan\ndated'}

In [149]:
text_sentence = "Most of the churches  <mask> on hand clapping and foot stomping as rhythmic accompaniment"
get_all_predictions(text_sentence)

Most of the churches  <mask> on hand clapping and foot stomping as rhythmic accompaniment


{'bart': 'rely\nrelied\ndepend\ndepended\nnow',
 'bert': 'relied\nfocused\nsettled\ncentered\ninsisted',
 'electra': 'rely\nfocus\ndepend\nrelied\ninsist',
 'roberta': 'relied\ndepended\ncarried\nkept\ninsisted',
 'xlm': 'are\nput\nturn\nwere\nsit',
 'xlnet': 'are\non\nup\nchurches\nis'}

In [150]:
text_sentence = "Most of the singing was <mask> a cappella "
get_all_predictions(text_sentence)

Most of the singing was <mask> a cappella 


{'bart': 'done\nperformed\nsung\neither\nin',
 'bert': 'sung\nperformed\nconducted\nmostly\ndone',
 'electra': 'in\non\nmade\njust\nfrom',
 'roberta': 'in\nsung\nsolo\ndelivered\nall',
 'xlm': 'on\nat\nin\nfrom\noff',
 'xlnet': 'a\nin\nan\nwas\njust'}

In [151]:
text_sentence = "The first  <mask> use of the term gospel song probably  <mask> in 1874"
get_all_predictions(text_sentence)

The first  <mask> use of the term gospel song probably  <mask> in 1874


{'bart': 'formal\nrecorded\nconscious\nwidespread\ndocumented',
 'bert': 'recorded\ndocumented\nknown\nsystematic\nwritten',
 'electra': 'known\npopular\ncommon\nrecorded\npublic',
 'roberta': 'formal\nrecorded\nwidespread\nproper\nlegal',
 'xlm': 'official\nknown\ncommon\npublic\ncommercial',
 'xlnet': 'hit\ntime\nsight\never\nin'}

In [152]:
text_sentence = "The original gospel songs were <mask> and composed by authors such as George F"
get_all_predictions(text_sentence)

The original gospel songs were <mask> and composed by authors such as George F


{'bart': 'written\ncomposed\nsung\npenned\nrecorded',
 'bert': 'written\ntranscribed\ncomposed\nsung\ncollected',
 'electra': 'written\ncomposed\nrecorded\nsung\nproduced',
 'roberta': 'written\nsung\ncomposed\ncollected\npenned',
 'xlm': 'written\nplanned\ninspired\ncreated\nsung',
 'xlnet': 'and\nwritten\nmade\ncomposed\nwas'}

In [153]:
text_sentence = "Southern gospel  <mask> all male, tenor-lead-baritone-bass quartet"
get_all_predictions(text_sentence)

Southern gospel  <mask> all male, tenor-lead-baritone-bass quartet


{'bart': 'singers\nsinger\nband\nmusicians\nmusic',
 'bert': 'quartet\nchoir\nmusic\nband\nensemble',
 'electra': 'includes\nhas\nis\nthe\nprovides',
 'roberta': "is\nan\ncomprises\nfeatures\n's",
 'xlm': 'to\nbenefits\nin\nfor\nis',
 'xlnet': 'all\ngospel\nchurch\nAll\nthe'}

In [154]:
text_sentence = "British black gospel  <mask> to Gospel music of the African diaspora"
get_all_predictions(text_sentence)

British black gospel  <mask> to Gospel music of the African diaspora


{'bart': 'singers\nmusic\nartists\nmusicians\nconcerts',
 'bert': 'refers\nrelates\nreferred\nrelating\nrelated',
 'electra': 'music\nrefers\ncontributed\nconnections\ncontributions',
 'roberta': 'fused\ndanced\ncontributed\nspread\nled',
 'xlm': 'links\ns\nled\nspoke\nties',
 'xlnet': 'to\ngospel\nand\nthe\nwith'}

In [155]:
text_sentence = "Into the Music is the 11th studio album by Northern Irish singer-songwriter Van Morrison, and was  <mask> in August 1979"
get_all_predictions(text_sentence)

Into the Music is the 11th studio album by Northern Irish singer-songwriter Van Morrison, and was  <mask> in August 1979


{'bart': 'released\nrecorded\ncommercially\nissued\noriginally',
 'bert': 'released\nissued\nrecorded\ncompleted\nlaunched',
 'electra': 'released\nrecorded\npublished\nannounced\nreissued',
 'roberta': 'released\nrecorded\nissued\ncompleted\npressed',
 'xlm': 'released\npublished\nproduced\nwritten\ncompleted',
 'xlnet': 'released\nmade\nissued\npublished\non'}

In [156]:
text_sentence = "The Music are an English alternative rock band,  <mask> in Kippax, Leeds in 1999"
get_all_predictions(text_sentence)

The Music are an English alternative rock band,  <mask> in Kippax, Leeds in 1999


{'bart': 'formed\nfounded\noriginally\nstarted\noriginating',
 'bert': 'formed\nforming\nbased\nfounded\nestablished',
 'electra': 'formed\nfounded\nestablished\nreleased\nstarted',
 'roberta': 'formed\nfounded\nborn\nforming\nbeginning',
 'xlm': 'established\nborn\nbased\nopened\ncreated',
 'xlnet': 'performed\nmade\nappeared\ndescribed\nand'}

In [157]:
text_sentence = "YouTube Music is a music streaming service  <mask> by YouTube, a subsidiary of Google"
get_all_predictions(text_sentence)

YouTube Music is a music streaming service  <mask> by YouTube, a subsidiary of Google


{'bart': 'offered\ndeveloped\noperated\nlaunched\nowned',
 'bert': 'provided\noffered\noperated\ncreated\npowered',
 'electra': 'provided\noffered\nowned\noperated\nrun',
 'roberta': 'owned\nprovided\nmanaged\ndeveloped\nlaunched',
 'xlm': 'offered\nrun\nprovided\ndeveloped\nestablished',
 'xlnet': 'provided\nservice\noffered\nmade\nserved'}

In [158]:
text_sentence = "It  <mask> a  tailored interface for the service"
get_all_predictions(text_sentence)

It  <mask> a  tailored interface for the service


{'bart': 'provides\ndesigned\ncreates\nalso\ndeveloped',
 'bert': 'provided\noffered\nprovides\nhad\nwas',
 'electra': 'provides\nprovided\nis\nwas\nhas',
 'roberta': 'created\ngave\nwas\nhad\nbuilt',
 'xlm': 'has\nprovides\noffers\nwas\nhad',
 'xlnet': 'it\nis\nIt\na\nin'}

In [159]:
text_sentence = "The service also  <mask> a premium tier"
get_all_predictions(text_sentence)

The service also  <mask> a premium tier


{'bart': 'offers\nhas\nincludes\noffered\nfeatures',
 'bert': 'offered\noffers\nadded\nfeatures\nfeatured',
 'electra': 'has\noffers\nfeatures\nincludes\nprovides',
 'roberta': 'has\noffers\ncarries\nfeatures\nhad',
 'xlm': 'has\nruns\nhad\nprovides\noffers',
 'xlnet': 'also\nas\nstill\nwith\non'}

In [160]:
text_sentence = "These subscription benefits are also <mask> to subscribers of Google Play Music and YouTube Premium"
get_all_predictions(text_sentence)

These subscription benefits are also <mask> to subscribers of Google Play Music and YouTube Premium


{'bart': 'available\nextended\noffered\nbeing\ncomplimentary',
 'bert': 'available\noffered\nprovided\nextended\naccessible',
 'electra': 'available\nprovided\naccessible\nextended\ngiven',
 'roberta': 'extended\navailable\nafforded\ngranted\noffered',
 'xlm': 'available\noffered\nopen\ngiven\nlimited',
 'xlnet': 'to\nnot\nonly\nalso\navailable'}

In [161]:
text_sentence = "In the 1960s, several British rock bands such as the Rolling Stones, the Who and the Animals were  <mask> to and promoted as being R&B bands"
get_all_predictions(text_sentence)

In the 1960s, several British rock bands such as the Rolling Stones, the Who and the Animals were  <mask> to and promoted as being R&B bands


{'bart': 'listened\nreferred\noften\nadded\ncompared',
 'bert': 'compared\nreferred\nlikened\nlistened\ndrawn',
 'electra': 'referred\nlinked\npromoted\nsigned\nopened',
 'roberta': 'referred\nattached\nsigned\nlistened\ntied',
 'xlm': 'reported\nmarried\nled\nsold\nup',
 'xlnet': 'referred\nalso\nto\ninitially\nall'}

In [162]:
text_sentence = "It  <mask> guest stars such as Ella Fitzgerald, John Davidson and Nelson Riddle and his orchestra"
get_all_predictions(text_sentence)

It  <mask> guest stars such as Ella Fitzgerald, John Davidson and Nelson Riddle and his orchestra


{'bart': 'featured\nwas\nattracted\noften\nstarred',
 'bert': 'featured\nincluded\nstarred\nfeatures\nhad',
 'electra': 'features\nhas\nincludes\nfeatured\nstars',
 'roberta': 'drew\nattracted\nbrought\nsaw\nfeatures',
 'xlm': 'had\nhas\nserved\nincludes\nsaw',
 'xlnet': 'guest\nis\nin\nit\nhas'}

In [163]:
text_sentence = "Music, Music is the eighth studio album by Australian-American pop singer Helen Reddy that was <mask> in the summer of 1976 by Capitol Records"
get_all_predictions(text_sentence)

Music, Music is the eighth studio album by Australian-American pop singer Helen Reddy that was <mask> in the summer of 1976 by Capitol Records


{'bart': 'released\nissued\nrecorded\npublished\ncommercially',
 'bert': 'released\nissued\nrecorded\nproduced\nreissued',
 'electra': 'released\nreissued\nrecorded\npublished\nsold',
 'roberta': 'released\nissued\npressed\nrecorded\nproduced',
 'xlm': 'released\nproduced\npublished\nannounced\nreported',
 'xlnet': 'released\nproduced\nrecorded\nmade\nissued'}

In [164]:
text_sentence = "It  <mask> on Billboard\\'s Top LP\\'s & Tapes chart that same month"
get_all_predictions(text_sentence)

It  <mask> on Billboard\'s Top LP\'s & Tapes chart that same month


{'bart': 'peaked\ndebuted\nranked\nwas\nspent',
 'bert': 'appeared\ndebuted\ncharted\nwas\nentered',
 'electra': 'appeared\ndebuted\nwas\npremiered\nwent',
 'roberta': 'landed\ndebuted\nappeared\nstayed\nwent',
 'xlm': 'appeared\nwas\ncame\nappears\nturned',
 'xlnet': '”\nOn\nIt\nchart\nThere'}

In [165]:
text_sentence = "The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums"
get_all_predictions(text_sentence)

The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums


{'bart': 'hailed\nacknowledged\nreleased\nchosen\nacclaimed',
 'bert': 'released\nrecorded\nselected\nissued\ndescribed',
 'electra': 'released\nrecorded\npublished\nannounced\ncredited',
 'roberta': 'chosen\nselected\npurchased\nbought\nsigned',
 'xlm': 'established\nidentified\ncreated\nmade\nproduced',
 'xlnet': 'released\nnamed\nlisted\nselected\nmarketed'}

In [166]:
text_sentence = "On August 23, 2005, Music, Music was  <mask> for the first time on compact disc as one of two Helen Reddy albums on one CD"
get_all_predictions(text_sentence)

On August 23, 2005, Music, Music was  <mask> for the first time on compact disc as one of two Helen Reddy albums on one CD


{'bart': 'released\nissued\navailable\npressed\nre',
 'bert': 'reissued\nreleased\nissued\navailable\ndistributed',
 'electra': 'released\nrecorded\nreissued\npublished\nremastered',
 'roberta': 'released\npressed\navailable\nplayed\npresented',
 'xlm': 'reported\nheard\nseen\nput\noffered',
 'xlnet': 'released\npresented\nannounced\navailable\nalso'}

In [167]:
text_sentence = "Music Choice is  <mask> nationwide by  \u202fComcast Xfinity,\u202fCharter Spectrum,\u202fCox Communications,\u202fVerizon Fios,\u202fDirecTV\u202fand many other mid-size and smaller  MVPDs"
get_all_predictions(text_sentence)

Music Choice is  <mask> nationwide by   Comcast Xfinity, Charter Spectrum, Cox Communications, Verizon Fios, DirecTV and many other mid-size and smaller  MVPDs


{'bart': 'carried\nnow\nbeing\noffered\ncurrently',
 'bert': 'offered\nprovided\ndistributed\nrecognized\nshared',
 'electra': 'accessed\noffered\navailable\ndistributed\nprovided',
 'roberta': 'administered\ncarried\nspread\nshared\ndistributed',
 'xlm': 'run\noffered\nreported\nserved\nbroadcast',
 'xlnet': 'provided\nalso\ns\nis\nmade'}

In [168]:
text_sentence = "music is a top-level domain name (TLD) <mask> in 2003 for use in music dissemination and appreciation"
get_all_predictions(text_sentence)

music is a top-level domain name (TLD) <mask> in 2003 for use in music dissemination and appreciation


{'bart': 'created\nreserved\nregistered\nlaunched\nassigned',
 'bert': 'created\nestablished\nlaunched\napproved\nregistered',
 'electra': 'established\ncreated\ndeveloped\nintroduced\nadopted',
 'roberta': 'registered\ncreated\nestablished\nlicensed\nfiled',
 'xlm': 'established\ncreated\nregistered\nbuilt\ndeveloped',
 'xlnet': 'created\nused\nnamed\nlaunched\nmade'}

In [169]:
text_sentence = "Limited  <mask> that their rights application had been approved by the Internet Corporation for Assigned Names and Numbers (ICANN)"
get_all_predictions(text_sentence)

Limited  <mask> that their rights application had been approved by the Internet Corporation for Assigned Names and Numbers (ICANN)


{'bart': 'companies\nliability\nparties\nCompanies\npartners',
 'bert': 'recognition\nnotice\nnews\nmention\nconcerns',
 'electra': 'notice\nstated\ninformation\nclaims\nnote',
 'roberta': 'proof\nshow\nconfirmation\nair\nrecognition',
 'xlm': 'reported\nrealized\nbelief\nsatisfied\nheard',
 'xlnet': 'Limited\nand\nsaid\nstated\nin'}

In [170]:
text_sentence = "K-pop (Korean: 케이팝; RR: keipap), short for Korean popular music, is a genre of music <mask> in South Korea as part of South Korean culture"
get_all_predictions(text_sentence)

K-pop (Korean: 케이팝; RR: keipap), short for Korean popular music, is a genre of music <mask> in South Korea as part of South Korean culture


{'bart': 'originating\nthat\npopular\noriginated\ndeveloped',
 'bert': 'popular\noriginating\ncommon\nproduced\nplayed',
 'electra': 'recorded\nproduced\npopular\nused\nplayed',
 'roberta': 'practiced\naccepted\ndeveloped\nadopted\npopular',
 'xlm': 'living\nfound\nestablished\ncreated\nmade',
 'xlnet': 'popular\nplayed\nperformed\nused\nin'}

In [171]:
text_sentence = "The term K-pop  <mask> popular in the 2000s"
get_all_predictions(text_sentence)

The term K-pop  <mask> popular in the 2000s


{'bart': 'became\nwas\nbegan\nhas\nculture',
 'bert': 'became\nwas\nremained\nis\ngrew',
 'electra': 'became\nwas\ngot\ngrew\nproved',
 'roberta': 'became\nwas\ngrew\nwent\ngot',
 'xlm': 'became\nwas\ngot\nturned\nvery',
 'xlnet': 'pop\npopular\npop\na\nmore'}

In [172]:
text_sentence = "The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums"
get_all_predictions(text_sentence)

The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums


{'bart': 'hailed\nacknowledged\nreleased\nchosen\nacclaimed',
 'bert': 'released\nrecorded\nselected\nissued\ndescribed',
 'electra': 'released\nrecorded\npublished\nannounced\ncredited',
 'roberta': 'chosen\nselected\npurchased\nbought\nsigned',
 'xlm': 'established\nidentified\ncreated\nmade\nproduced',
 'xlnet': 'released\nnamed\nlisted\nselected\nmarketed'}

In [173]:
text_sentence = "The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums"
get_all_predictions(text_sentence)

The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums


{'bart': 'hailed\nacknowledged\nreleased\nchosen\nacclaimed',
 'bert': 'released\nrecorded\nselected\nissued\ndescribed',
 'electra': 'released\nrecorded\npublished\nannounced\ncredited',
 'roberta': 'chosen\nselected\npurchased\nbought\nsigned',
 'xlm': 'established\nidentified\ncreated\nmade\nproduced',
 'xlnet': 'released\nnamed\nlisted\nselected\nmarketed'}

In [174]:
text_sentence = "The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums"
get_all_predictions(text_sentence)

The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums


{'bart': 'hailed\nacknowledged\nreleased\nchosen\nacclaimed',
 'bert': 'released\nrecorded\nselected\nissued\ndescribed',
 'electra': 'released\nrecorded\npublished\nannounced\ncredited',
 'roberta': 'chosen\nselected\npurchased\nbought\nsigned',
 'xlm': 'established\nidentified\ncreated\nmade\nproduced',
 'xlnet': 'released\nnamed\nlisted\nselected\nmarketed'}

In [175]:
text_sentence = "The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums"
get_all_predictions(text_sentence)

The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums


{'bart': 'hailed\nacknowledged\nreleased\nchosen\nacclaimed',
 'bert': 'released\nrecorded\nselected\nissued\ndescribed',
 'electra': 'released\nrecorded\npublished\nannounced\ncredited',
 'roberta': 'chosen\nselected\npurchased\nbought\nsigned',
 'xlm': 'established\nidentified\ncreated\nmade\nproduced',
 'xlnet': 'released\nnamed\nlisted\nselected\nmarketed'}

In [176]:
text_sentence = "The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums"
get_all_predictions(text_sentence)

The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums


{'bart': 'hailed\nacknowledged\nreleased\nchosen\nacclaimed',
 'bert': 'released\nrecorded\nselected\nissued\ndescribed',
 'electra': 'released\nrecorded\npublished\nannounced\ncredited',
 'roberta': 'chosen\nselected\npurchased\nbought\nsigned',
 'xlm': 'established\nidentified\ncreated\nmade\nproduced',
 'xlnet': 'released\nnamed\nlisted\nselected\nmarketed'}